In [60]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.stats import multivariate_normal
import os

Initializing the variables- mean, variances and weights. As pixel scale goes from 0-255, we choose mean around 125. Also, as sum of weights should be 1, we initalize them with same values. Chose a random value of 30 for variances.

In [33]:
def initializer(height,width,n_gauss,threshold,lrate):
    means=np.zeros((height,width,n_gauss,3) )
    variances=np.zeros((height,width,n_gauss))
    weights=np.zeros((height,width,n_gauss))
    for h in range(height):
        for w in range(width):
            means[h,w]=np.array([[120, 120, 120]]*n_gauss)
            weights[h,w]=[1.0/n_gauss]*n_gauss
            variances[h,w]=[30.0]*n_gauss
    return means, variances, weights
    

This function sorts the Gaussian models GMs by the ratio of weight/SD. Then it finds if sum of weights of first b is greater than the threshold. If yes, that means that the GM is in the background model. 

In [ ]:
def sorter(height,width,n_gauss,threshold,means, variances, weights):
    #intialize status matrix
    B=np.zeros((height,width))
    for h in range(height):
        for w in range(width):
            B[h,w]=-1
            ratios=[0,0,0]
            for k in range(n_gauss):
                #ratio for kth GM is given by weight/ SD
                ratios[k]=(weights[h,w,k]/np.sqrt(variances[h,w,k]))
                #sort the GMs by ratios and reverse it as the list will be ascending sorted while we need descending order.
                indices=np.array(np.argsort(ratios)[::-1])
                #arrange the corresponding means variances and weights of the GMs in sorted order.
                means[h,w,:]=means[h,w,[indices]]
                variances[h,w,:]=variances[h,w,[indices]]
                weights[h,w,:]=weights[h,w,[indices]]
                sum_weights=0
                
                for l in range(n_gauss):
                    sum_weights=weights[h,w,l]
                    #checks if the sum of the weights of the GMs is above the threshold.
                    if sum_weights>=threshold:
                        B[h,w]=l
                        break
                #if no background detected,make last GM as foreground.
                if B[h,w]==-1:
                    B[h,w]=n_gauss-2
        return B

This function updates the models according to the current intensity xt. 

In [53]:
def updates(frame, B,height,width,lrate,means, weights,variances):
        #initalize label matrix for every pixel
        classes=np.zeros((height,width))
        for h in range(height):
            for w in range(width):
                #current frame of the video
                X=frame[h,w]
                M=-1
                for k in range(n_gauss):
                    #inverse of the covariance matrix. Assuming that all channels are independent, so that C is an identity matrix.
                    C_inv=np.linalg.inv(variances[h,w,k]*np.eye(3))
                    #subtracting mean from X: X-u
                    X_normalized=X-means[h,w,k]
                    #finding the Mahalonobis distance square
                    distance=np.dot(X_normalized.T, np.dot(C_inv, X_normalized))
                    #if distance is below 2.5^2 * variance sigma^2, a atch with a GM of index l is found
                    if distance<6.25*variances[h,w,k]:
                        M=k
                        break
                #Update parameters if match is found
                if M!=-1:  
                    weights[h,w]=(1.0-lrate)*weights[h,w]
                    weights[h,w,M]+=lrate
                    R=lrate * multivariate_normal.pdf(X,means[h,w,M],np.linalg.inv(C_inv))
                    variances[M]=(1.0-R)*variances[h,w,M]+R*np.dot((X-means[h,w,M]).T, (X-means[h,w,M]))
                    means[h,w,M]=(1.0-R)*means[h,w,M]+R*X
                    #label the current x pixel
                    if M>B[h,w]:
                        classes[h,w]=250
                else:
                    means[h,w,-1]=X
                    classes[h,w]=250
        return classes

Test function to analyse input frames and create output.

In [83]:
def test(height,width,n_gauss,lrate,threshold,weights,means,variances):
        #load the test video
        cap=cv2.VideoCapture(r'C:/Users/Admin/Downloads/videoplayback_Trim_new.mp4')
        cap.set(3,width)
        cap.set(4,height)
        n_frame=0
        #create output video
        out = cv2.VideoWriter('C:/Users/Admin/Downloads/projectnewnew.avi',cv2.VideoWriter_fourcc(*'XVID'), 10, (height,width))
        retval=1
        while retval:
            #capture a frame from the video
            retval,frame=cap.read()
            n_frame+=1
            print("current frame: ", n_frame)
            B=sorter(height,width,n_gauss,threshold,means, variances, weights)
            classes=updates(frame, B,height,width,lrate,means, weights,variances)
            #write the output frame to the video
            out.write(classes)
            #also save the output frame
            cv2.imwrite( "frame{}.png".format(n_frame), classes );
        out.release()
        cap.release()
        cv2.destroyAllWindows()

In [84]:
height=240
width=320
lrate=0.02
n_gauss=3
threshold=0.9
means, variances, weights=initializer(height,width,n_gauss,threshold,lrate)

In [ ]:
test(height,width,n_gauss,lrate,threshold,weights,means,variances)

current frame:  1
current frame:  2
current frame:  3
current frame:  4
current frame:  5
current frame:  6
current frame:  7
current frame:  8
current frame:  9
current frame:  10
current frame:  11
current frame:  12
current frame:  13
current frame:  14
current frame:  15
current frame:  16
current frame:  17
current frame:  18
current frame:  19
current frame:  20
current frame:  21
current frame:  22
current frame:  23
current frame:  24
current frame:  25
current frame:  26
current frame:  27
current frame:  28
current frame:  29
current frame:  30
current frame:  31
current frame:  32
current frame:  33
current frame:  34
current frame:  35
current frame:  36
current frame:  37
current frame:  38
current frame:  39
current frame:  40
current frame:  41
current frame:  42
current frame:  43
current frame:  44
current frame:  45
current frame:  46
current frame:  47
current frame:  48
current frame:  49
current frame:  50
current frame:  51
current frame:  52
current frame:  53
cu